# kaggle username and api key to env params and dataset download

In [1]:
import os
import json

In [ ]:
with open("/Users/arsknz/Downloads/kaggle.json") as f:
    us_key = json.load(f)

os.environ["KAGGLE_USERNAME"] = us_key["username"]
os.environ["KAGGLE_KEY"] = us_key["key"]

!kaggle datasets download -d gabrielrichter/sales-by-products-assortment  -p ~/osnov/code/sbor_data/ --force
!unzip sales-by-products-assortment.zip
